# Прогнозирование на основе обученной модели классификации базы контрактов 
# (Prediction based on a trained contract base classification model)

### 1. Подготовка и обработка датасета (Preparation and preprocessing of dataset)

#### Обработка массива данных по номенклатуре (Preprocessing the sku dataset)

In [1]:
# standart library block 
import pandas as pd
import numpy as np

In [2]:
# Чтение файла с данными по sku (Reading sku data file)
skuData = pd.read_csv("Dataset/1_skuData.csv", dtype={'Id': object, 'okpd12': object}, sep=';')

In [3]:
# Информация о датасете (Info about dataset)
skuData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10497862 entries, 0 to 10497861
Data columns (total 5 columns):
Id          object
okpd12      object
skuPrice    float64
skuQt       float64
skuSum      float64
dtypes: float64(3), object(2)
memory usage: 400.5+ MB


In [4]:
# Создание новой переменной с помощью нарезки 'okpd12' до 2-го элемента
# (Create new feature with help slicing 'okpd12' till 2nd element)
skuData['okpd1']= skuData['okpd12'].str.slice(0, 2, 1)

In [5]:
# Группировка данных по id и транспонирование их по ОКПД (Grouping data by id and transposing them by OKPD)
Dataset_sku = pd.pivot_table(skuData,
                             index=["Id"],
                             columns=["okpd1"], 
                             values=["skuPrice", "skuQt", "skuSum"], 
                             aggfunc={"skuPrice":[max,min,np.mean], 
                                      "skuQt":[max,min,np.mean, np.sum], 
                                      "skuSum":[max,min,np.mean, np.sum]}, 
                             fill_value=0)

In [6]:
# Сброс индексов и переименование столбцов (Reset index and rename columns)
Dataset_sku.reset_index()
headers = list(range(len(Dataset_sku.columns)))
Dataset_sku.columns = headers

In [7]:
# Размер датасета (Dataset size)
Dataset_sku.shape

(248653, 979)

#### Обработка массива данных по контрактам (Preprocessing the contracts dataset)

In [8]:
# Чтение файлов с данными по агрегированным переменным контактов и значению таргета (Reading contracts data file)
countsData = pd.read_csv("Dataset/2_countsData.csv", dtype={'Id': object}, sep=';')
sumData = pd.read_csv("Dataset/3_sumData.csv", dtype={'Id': object}, sep=';')
tipData = pd.read_csv("Dataset/4_tipData.csv", dtype={'Id': object}, sep=';')
targetData = pd.read_csv("Dataset/5_targetData.csv", dtype={'Id': object}, sep=';')

In [9]:
countsData = countsData.drop('sku_quant', axis=1)

In [10]:
# Объединение файлов в общий массив Dataset_contracts (Joining files into a common dataset Dataset_contracts)
count_sumData = pd.merge(countsData, 
                         sumData, 
                         on = 'Id')
count_sum_tipData = pd.merge(count_sumData,
                            tipData,
                            on = 'Id')
Dataset_contracts = pd.merge(count_sum_tipData,
                            targetData,
                            on = 'Id')

In [11]:
Dataset_contracts.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 248653 entries, 0 to 248652
Data columns (total 15 columns):
Id                    248653 non-null object
contract_quant        248653 non-null int64
cnt_uniq_buyer        248653 non-null int64
cnt_buyRegion         248653 non-null int64
cnt_buy_contPerson    248653 non-null int64
skuOKPD_quant         248653 non-null int64
cnt_OKEI              248653 non-null int64
personal              248653 non-null int64
cnt_months            248653 non-null int64
tot_contractSum       248653 non-null float64
min_contractSum       248653 non-null float64
max_contractSum       248653 non-null float64
avg_contractSum       248653 non-null float64
bidding_type          248653 non-null int64
soft_target           248653 non-null int64
dtypes: float64(4), int64(10), object(1)
memory usage: 30.4+ MB


#### Создание общего массива данных для обработки с помощью обученной модели классификации (Create common dataset for processing using a trained model)

In [12]:
# Создание общего датасета для обучения модели (Create a common dataset for machine learning)
kz_data = pd.merge(Dataset_contracts, 
                   Dataset_sku, 
                   on = 'Id')

In [13]:
# Размер итогового датасета (Dataset size)
kz_data.shape

(248653, 994)

### 2. Прогнозирование на основе обученной модели (Prediction based on a trained model)

In [14]:
# standart library block
import numpy as np
import pandas as pd
import pickle

# learning library block
from sklearn.preprocessing import StandardScaler
from sklearn.externals import joblib

from catboost import CatBoostClassifier
from catboost import cv
from catboost.datasets import amazon
from catboost.datasets import epsilon

#### Подготовка датасета (Prepared dataset)

In [15]:
# Определение features. 
# (Purpose of "features".)
                     #y=kz_data['soft_target'].copy()
X=kz_data.drop('soft_target', axis=1)

In [16]:
# Стандартизация значений переменных (Standardization of features values)
scaler = StandardScaler()
scaler.fit_transform(X)
X_scaled = scaler.transform(X)

C:\Users\voropaev.om\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64, float64, object were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\voropaev.om\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\base.py:464: DataConversionWarning: Data with input dtype int64, float64, object were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
C:\Users\voropaev.om\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: Data with input dtype int64, float64, object were all converted to float64 by StandardScaler.
  after removing the cwd from sys.path.


#### Прогнозирование - Модель CatBoost v.4 (Prediction - Model CatBoost v.4)

In [17]:
model = CatBoostClassifier() # выбираем тип модели (choose the type of model)
model.load_model('Trained models/kz_cat4.cbm') # загружаем обученную модель (load trained model)

kz_pred_cat_train = model.predict_proba(data=X)[:,1] # предсказание для рабочего датасета (prediction for work sample)

In [18]:
# Сохранение полученных резульатов (Save the results)
df1 = pd.DataFrame(X['Id'])
df2 = pd.DataFrame(kz_pred_cat_train)
df2.columns=['Predictions']
res = pd.concat([df1, df2.reindex(df1.index)], axis=1)
predict_results = pd.DataFrame(res).to_csv('Results prediction/prediction_kz_model.csv', index=False, sep=';')